In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize

# функция Розенброка
def rosen(x):
    return (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

# запись траектории
path = []
def callback(xk):
    path.append(xk.copy())

res = minimize(
    rosen,
    x0=np.array([-1.5, 1.5]),
    method="L-BFGS-B",
    callback=callback
)

path = np.array(path)

# сетка для контуров
x = np.linspace(-2, 2, 300)
y = np.linspace(-1, 3, 300)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100*(Y - X**2)**2

frames = [
    go.Frame(
        data=[go.Scatter(
            x=path[:k, 0],
            y=path[:k, 1],
            mode="lines+markers"
        )]
    )
    for k in range(1, len(path))
]

fig = go.Figure(
    data=[
        go.Contour(x=x, y=y, z=Z, colorscale="Viridis"),
        go.Scatter(x=[path[0,0]], y=[path[0,1]], mode="lines+markers")
    ],
    frames=frames
)

fig.update_layout(
    title="Геометрическая интерпретация L-BFGS",
    xaxis_title="w₁",
    yaxis_title="w₂",
    updatemenus=[{
        "type": "buttons",
        "buttons": [{
            "label": "▶ Play",
            "method": "animate",
            "args": [None]
        }]
    }]
)

fig.show()


In [ ]:
import numpy as np
from scipy.optimize import minimize
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ─── Функция Розенброка ───────────────────────────────────────
def rosen(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2


def rosen_grad(x):
    return np.array([
        -2*(1 - x[0]) - 400*x[0]*(x[1] - x[0]**2),
         200*(x[1] - x[0]**2)
    ])


# ─── Сбор траектории через callback ───────────────────────────
path = []

def callback(xk):
    path.append(xk.copy())


# ─── Запуск оптимизации ───────────────────────────────────────
x0 = np.array([-1.5, 1.1])
res = minimize(
    rosen, x0,
    method='L-BFGS-B',
    jac=rosen_grad,
    callback=callback,
    options={'maxiter': 120, 'disp': False}
)

path = np.array(path)
if len(path) == 0 or np.linalg.norm(path[-1] - res.x) > 1e-4:
    path = np.vstack([path, res.x])

# print(f"Найденная точка: {res.x:.6f}")
print(f"Значение функции: {res.fun:.2e}")
print(f"Количество итераций: {len(path)-1}")

# ─── Подготовка данных для контуров ───────────────────────────
x = np.linspace(-2.0, 2.0, 180)
y = np.linspace(-0.5, 2.5, 180)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100 * (Y - X**2)**2

# Логарифмируем для лучшей визуализации уровней
Z_log = np.log1p(Z)

# ─── Создаём анимированный график ─────────────────────────────
fig = go.Figure()

# 1. Статичные контуры (log-scale)
fig.add_trace(
    go.Contour(
        x=x, y=y, z=Z_log,
        colorscale='Viridis',
        contours=dict(
            start=0, end=np.log1p(Z.max()), size=0.4,
            showlabels=True, labelfont=dict(size=10)
        ),
        showscale=True,
        colorbar=dict(title="log(1 + f)"),
        name="Уровни"
    )
)

# 2. Полная траектория (тонкая серая линия, всегда видна)
fig.add_trace(
    go.Scatter(
        x=path[:,0], y=path[:,1],
        mode='lines',
        line=dict(color='rgba(80,80,80,0.5)', width=1.5),
        name='Полный путь'
    )
)

# 3. Текущая точка (будет анимироваться)
fig.add_trace(
    go.Scatter(
        x=[path[0,0]], y=[path[0,1]],
        mode='markers+text',
        marker=dict(size=12, color='red', symbol='circle', line=dict(width=2, color='darkred')),
        text=['start'],
        textposition='top center',
        name='Текущая точка'
    )
)

# ─── Кадры анимации ───────────────────────────────────────────
frames = []
for i in range(1, len(path)):
    frame = go.Frame(
        data=[
            # обновляем только третью трассу (точку)
            go.Scatter(
                x=[path[i,0]], y=[path[i,1]],
                mode='markers+text',
                marker=dict(size=12, color='red', symbol='circle', line=dict(width=2, color='darkred')),
                text=[f'#{i}'],
                textposition='top center'
            )
        ],
        traces=[2],   # индекс трассы, которую обновляем
        name=f'frame{i}'
    )
    frames.append(frame)

fig.frames = frames

# ─── Настройки анимации и вида ────────────────────────────────
fig.update_layout(
    title="L-BFGS на функции Розенброка — геометрия шагов",
    width=780, height=680,
    xaxis_title="x₁", yaxis_title="x₂",
    showlegend=True,
    updatemenus=[dict(
        type="buttons",
        buttons=[
            dict(label="Play",
                 method="animate",
                 args=[None, {
                     "frame": {"duration": 400, "redraw": True},
                     "fromcurrent": True,
                     "transition": {"duration": 150},
                     "mode": "immediate"
                 }]),
            dict(label="Pause",
                 method="animate",
                 args=[[None], {
                     "frame": {"duration": 0, "redraw": False},
                     "mode": "immediate"
                 }])
        ],
        showactive=True,
        x=0.01, y=1.05
    )],
    # начальные пределы осей
    xaxis=dict(range=[-1.8, 1.8]),
    yaxis=dict(range=[-0.2, 2.2])
)

fig.show()

In [ ]:
import numpy as np
from scipy.optimize import minimize
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ─── Rosenbrock ────────────────────────────────────────────────
def rosen(x):
    return (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

def rosen_grad(x):
    return np.array([
        -2*(1 - x[0]) - 400*x[0]*(x[1] - x[0]**2),
        200*(x[1] - x[0]**2)
    ])

# ─── Сохраняем историю для последующего воспроизведения ───────
history = []           # список словарей: x, grad, s, y

def callback(xk):
    if len(history) == 0:
        history.append({"x": xk.copy(), "grad": rosen_grad(xk)})
    else:
        prev = history[-1]
        s = xk - prev["x"]
        y = rosen_grad(xk) - prev["grad"]
        history.append({
            "x": xk.copy(),
            "grad": rosen_grad(xk),
            "s": s,
            "y": y
        })

x0 = np.array([-1.2, 1.0])
res = minimize(rosen, x0, method='L-BFGS-B',
               jac=rosen_grad, callback=callback,
               options={'maxiter': 80, 'ftol':1e-9})

print(f"Итераций: {len(history)-1},  точка: {res.x}")

# ─── Функция двухмерного произведения H @ v  (L-BFGS two-loop recursion) ──
def lbfgs_matvec(v, hist, m=8, init_scale=None):
    """ Приближает H_k @ v   (H — аппроксимация обратного гессиана) """
    if len(hist) < 2:
        if init_scale is None:
            return v * 0.1          # очень осторожный шаг
        return v * init_scale

    # берём последние m пар
    pairs = hist[-m:]
    s_list = [p["s"] for p in pairs if "s" in p]
    y_list = [p["y"] for p in pairs if "y" in p]
    if not s_list:
        return v * 0.1

    # начальное приближение H0
    last_y, last_s = y_list[-1], s_list[-1]
    gamma = np.dot(last_s, last_y) / np.dot(last_y, last_y)
    q = v.copy()
    alpha = []

    # прямой проход
    for si, yi in zip(reversed(s_list), reversed(y_list)):
        a = np.dot(si, q) / np.dot(si, yi)
        alpha.append(a)
        q -= a * yi

    r = q * gamma

    # обратный проход
    for si, yi, a in zip(s_list[::-1], y_list[::-1], alpha[::-1]):
        beta = np.dot(yi, r) / np.dot(si, yi)
        r += (a - beta) * si

    return r

# ─── Сетка для визуализации квадратичной модели ───────────────
xx = np.linspace(-1.8, 1.8, 90)
yy = np.linspace(-0.4, 2.2, 90)
XX, YY = np.meshgrid(xx, yy)

# ─── Plotly фигура ─────────────────────────────────────────────
fig = make_subplots(rows=1, cols=1)

# Истинная функция (лог-уровни)
Z_true = np.log1p(rosen([XX, YY]))
fig.add_trace(
    go.Contour(
        x=xx, y=yy, z=Z_true,
        colorscale='Viridis', opacity=0.45,
        contours_coloring='lines',
        line_width=0.8, showscale=False,
        name='Уровни f'
    )
)

# Будем добавлять эллипсы квадратичной модели
frames = []
visible_traces = []  # для управления видимостью

for k in range(len(history)):
    xk = history[k]["x"]
    gradk = history[k]["grad"]

    # квадратичная модель относительно xk
    def quad_model(p):
        direction = p - xk
        Hv = lbfgs_matvec(direction, history[:k+1], m=12)
        return 0.5 * np.dot(direction, Hv) + np.dot(gradk, direction)

    Z_model = np.array([[quad_model(np.array([xi,yi])) for xi in xx] for yi in yy])
    Z_model = np.log1p(np.maximum(Z_model, 0))  # для визуализации

    # добавляем контур модели
    cont = go.Contour(
        x=xx, y=yy, z=Z_model,
        colorscale='RdBu', opacity=0.7,
        contours=dict(start=0, end=3, size=0.3),
        showscale=False,
        visible=(k==0),  # только первый видим сразу
        name=f'model k={k}'
    )
    fig.add_trace(cont)

    # текущая точка
    pt = go.Scatter(
        x=[xk[0]], y=[xk[1]],
        mode='markers+text',
        marker=dict(size=10, color='black', symbol='x'),
        text=[f'k={k}'],
        textposition='top center',
        visible=(k==0)
    )
    fig.add_trace(pt)

    # кадр
    frame_data = []
    for i in range(2, len(fig.data)):  # пропускаем фон
        visible = [False] * len(fig.data)
        visible[0] = True           # истинная функция всегда
        if i == 2*k + 2: visible[i] = True     # контур модели
        if i == 2*k + 3: visible[i] = True     # точка
        frame_data.append(go.Frame(
            data=[go.Contour(visible=vis) if 'Contour' in str(d) else go.Scatter(visible=vis)
                  for d,vis in zip(fig.data, visible)],
            traces=list(range(len(fig.data))),
            name=f'frame{k}'
        ))

frames = [go.Frame(
    data=[d if i==0 or i==2*k+2 or i==2*k+3 else go.Scatter(visible=False) for i,d in enumerate(fig.data)],
    traces=list(range(len(fig.data))),
    name=f'k={k}'
) for k in range(len(history))]

fig.frames = frames

# начальная видимость
for trace in fig.data:
    trace.visible = False
fig.data[0].visible = True   # фон
fig.data[2].visible = True   # первая модель
fig.data[3].visible = True   # первая точка

fig.update_layout(
    title="Эволюция квадратичной аппроксимации в L-BFGS<br>(красные/синие — уровни модели m_k, фон — уровни настоящей f)",
    width=820, height=720,
    xaxis_title="x", yaxis_title="y",
    updatemenus=[dict(
        buttons=[
            dict(label="Play", method="animate",
                 args=[None, {"frame": {"duration": 900, "redraw": True},
                              "fromcurrent": True, "mode": "immediate"}]),
            dict(label="Pause", method="animate",
                 args=[[None], {"frame": {"duration": 0, "redraw": False},
                                "mode": "immediate"}])
        ],
        showactive=True
    )],
    showlegend=False
)

fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# ---------- функция и градиент ----------
a, b = 1.0, 5.0

def f(x):
    return 0.5 * (a * x[0]**2 + b * x[1]**2)

def grad(x):
    return np.array([a * x[0], b * x[1]])

# истинный Гессиан
H_true = np.array([[a, 0],
                   [0, b]])

# ---------- BFGS ----------
def bfgs_update(B, s, y):
    rho = 1.0 / (y @ s)
    I = np.eye(len(s))
    return (I - rho * np.outer(s, y)) @ B @ (I - rho * np.outer(y, s)) + rho * np.outer(s, s)

# ---------- оптимизация ----------
x = np.array([2.0, 2.0])
B = np.eye(2)

Bs = [B.copy()]  # сохраняем аппроксимации

for _ in range(5):
    p = -np.linalg.solve(B, grad(x))
    x_new = x + 0.3 * p

    s = x_new - x
    y = grad(x_new) - grad(x)

    B = bfgs_update(B, s, y)

    Bs.append(B.copy())
    x = x_new

# ---------- сетка ----------
X = np.linspace(-2, 2, 60)
Y = np.linspace(-2, 2, 60)
XX, YY = np.meshgrid(X, Y)

def quad_surface(B):
    Z = np.zeros_like(XX)
    for i in range(XX.shape[0]):
        for j in range(XX.shape[1]):
            v = np.array([XX[i,j], YY[i,j]])
            Z[i,j] = 0.5 * v @ B @ v
    return Z

# ---------- plotly ----------
fig = go.Figure()

# истинный Гессиан
fig.add_surface(
    x=XX, y=YY, z=quad_surface(H_true),
    opacity=0.35,
    colorscale="Viridis",
    name="True Hessian"
)

# аппроксимации
for k, Bk in enumerate(Bs):
    fig.add_surface(
        x=XX, y=YY, z=quad_surface(Bk),
        opacity=0.15,
        showscale=False,
        name=f"BFGS iter {k}"
    )

fig.update_layout(
    title="Approximation of the Hessian matrix (BFGS)",
    scene=dict(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="Quadratic form"
    ),
    width=900,
    height=700
)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as sp
from scipy.optimize import approx_fprime

def approx_hessian(f, x0, epsilon=1e-6):
    """
    Аппроксимация матрицы Гессе с помощью конечных разностей
    
    Parameters:
    -----------
    f : callable
        Функция от n переменных, возвращает скаляр
    x0 : array_like
        Точка, в которой вычисляется гессиан
    epsilon : float
        Шаг для конечных разностей
    
    Returns:
    --------
    hess : ndarray
        Матрица Гессе размером n x n
    """
    n = len(x0)
    hess = np.zeros((n, n))
    
    # Базовая точка
    f0 = f(x0)
    
    # Диагональные элементы (вторая производная)
    for i in range(n):
        # Вперед
        x_plus = x0.copy()
        x_plus[i] += epsilon
        f_plus = f(x_plus)
        
        # Назад
        x_minus = x0.copy()
        x_minus[i] -= epsilon
        f_minus = f(x_minus)
        
        # Центральная разность для второй производной
        hess[i, i] = (f_plus - 2*f0 + f_minus) / (epsilon**2)
    
    # Смешанные производные
    for i in range(n):
        for j in range(i+1, n):
            # Четыре точки для смешанной производной
            x_plus_plus = x0.copy()
            x_plus_plus[i] += epsilon
            x_plus_plus[j] += epsilon
            
            x_plus_minus = x0.copy()
            x_plus_minus[i] += epsilon
            x_plus_minus[j] -= epsilon
            
            x_minus_plus = x0.copy()
            x_minus_plus[i] -= epsilon
            x_minus_plus[j] += epsilon
            
            x_minus_minus = x0.copy()
            x_minus_minus[i] -= epsilon
            x_minus_minus[j] -= epsilon
            
            # Смешанная производная
            f_pp = f(x_plus_plus)
            f_pm = f(x_plus_minus)
            f_mp = f(x_minus_plus)
            f_mm = f(x_minus_minus)
            
            hess[i, j] = (f_pp - f_pm - f_mp + f_mm) / (4 * epsilon**2)
            hess[j, i] = hess[i, j]  # Симметрия
    
    return hess

def quadratic_form(x, A):
    """Квадратичная форма: f(x) = x^T A x"""
    return x.T @ A @ x

def rosenbrock(x, a=1, b=100):
    """Функция Розенброка (тестовая функция)"""
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2

def visualize_function_and_hessian(f, x_range=(-2, 2), y_range=(-2, 2), 
                                 point=(0, 0), resolution=50):
    """
    Визуализация функции и аппроксимации Гессиана в точке
    
    Parameters:
    -----------
    f : callable
        Функция двух переменных f(x, y)
    x_range : tuple
        Диапазон для оси X
    y_range : tuple
        Диапазон для оси Y
    point : tuple
        Точка (x, y) для анализа
    resolution : int
        Разрешение сетки
    """
    # Создаем сетку
    x = np.linspace(x_range[0], x_range[1], resolution)
    y = np.linspace(y_range[0], y_range[1], resolution)
    X, Y = np.meshgrid(x, y)
    
    # Вычисляем значения функции
    Z = np.zeros_like(X)
    for i in range(resolution):
        for j in range(resolution):
            Z[i, j] = f([X[i, j], Y[i, j]])
    
    # Аппроксимируем гессиан в точке
    point_array = np.array(point)
    hessian = approx_hessian(f, point_array)
    
    # Вычисляем квадратичную аппроксимацию
    f0 = f(point_array)
    grad = approx_fprime(point_array, f, 1e-6)
    
    # Квадратичная аппроксимация: f(x) ≈ f(p) + ∇f(p)·(x-p) + 0.5*(x-p)ᵀH(x-p)
    def quadratic_approx(xy):
        dx = np.array(xy) - point_array
        return f0 + grad @ dx + 0.5 * dx.T @ hessian @ dx
    
    Z_approx = np.zeros_like(X)
    for i in range(resolution):
        for j in range(resolution):
            Z_approx[i, j] = quadratic_approx([X[i, j], Y[i, j]])
    
    # Создаем subplot
    fig = sp.make_subplots(
        rows=2, cols=2,
        subplot_titles=('Исходная функция', 
                       'Квадратичная аппроксимация',
                       'Разность: Функция - Аппроксимация',
                       'Поверхности вблизи точки'),
        specs=[[{'type': 'surface'}, {'type': 'surface'}],
               [{'type': 'surface'}, {'type': 'surface'}]]
    )
    
    # 1. Исходная функция
    fig.add_trace(
        go.Surface(x=X, y=Y, z=Z, colorscale='Viridis', 
                  opacity=0.9, showscale=False,
                  contours=dict(z=dict(show=True, project_z=True))),
        row=1, col=1
    )
    
    # 2. Квадратичная аппроксимация
    fig.add_trace(
        go.Surface(x=X, y=Y, z=Z_approx, colorscale='Plasma',
                  opacity=0.9, showscale=False,
                  contours=dict(z=dict(show=True, project_z=True))),
        row=1, col=2
    )
    
    # 3. Разность
    Z_diff = Z - Z_approx
    fig.add_trace(
        go.Surface(x=X, y=Y, z=Z_diff, colorscale='RdBu',
                  opacity=0.9, showscale=True,
                  contours=dict(z=dict(show=True))),
        row=2, col=1
    )
    
    # 4. Детальный вид вблизи точки
    zoom_range = 0.5
    x_zoom = np.linspace(point[0]-zoom_range, point[0]+zoom_range, 30)
    y_zoom = np.linspace(point[1]-zoom_range, point[1]+zoom_range, 30)
    X_zoom, Y_zoom = np.meshgrid(x_zoom, y_zoom)
    
    Z_zoom = np.zeros_like(X_zoom)
    Z_approx_zoom = np.zeros_like(X_zoom)
    for i in range(30):
        for j in range(30):
            Z_zoom[i, j] = f([X_zoom[i, j], Y_zoom[i, j]])
            Z_approx_zoom[i, j] = quadratic_approx([X_zoom[i, j], Y_zoom[i, j]])
    
    fig.add_trace(
        go.Surface(x=X_zoom, y=Y_zoom, z=Z_zoom, 
                  colorscale='Viridis', opacity=0.7,
                  name='Функция'),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Surface(x=X_zoom, y=Y_zoom, z=Z_approx_zoom,
                  colorscale='Plasma', opacity=0.7,
                  name='Аппроксимация'),
        row=2, col=2
    )
    
    # Добавляем точку анализа
    for row in [1, 2]:
        for col in [1, 2]:
            fig.add_trace(
                go.Scatter3d(x=[point[0]], y=[point[1]], z=[f0],
                           mode='markers', marker=dict(size=5, color='red'),
                           name='Точка анализа'),
                row=row, col=col
            )
    
    # Обновляем layout
    fig.update_layout(
        title=f'Аппроксимация матрицы Гессе в точке {point}',
        scene=dict(aspectmode='cube'),
        scene2=dict(aspectmode='cube'),
        scene3=dict(aspectmode='cube'),
        scene4=dict(aspectmode='cube'),
        height=1000,
        showlegend=True
    )
    
    # Обновляем подписи осей
    scenes = ['scene', 'scene2', 'scene3', 'scene4']
    for scene in scenes:
        fig.layout[scene].update(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='f(x,y)'
        )
    
    fig.show()
    
    # Выводим информацию о гессиане
    print("=" * 60)
    print(f"АНАЛИЗ В ТОЧКЕ: {point}")
    print("=" * 60)
    print(f"Значение функции: f{point} = {f0:.6f}")
    print(f"\nГрадиент (аппроксимированный):")
    print(grad)
    print(f"\nМатрица Гессе (аппроксимированная):")
    print(hessian)
    print(f"\nСобственные значения гессиана:")
    eigenvalues = np.linalg.eigvals(hessian)
    print(eigenvalues)
    
    # Анализ критической точки
    if np.all(np.abs(grad) < 1e-6):
        print("\n✱ Точка является КРИТИЧЕСКОЙ (градиент ≈ 0)")
        if np.all(eigenvalues > 0):
            print("  → Локальный МИНИМУМ (гессиан положительно определён)")
        elif np.all(eigenvalues < 0):
            print("  → Локальный МАКСИМУМ (гессиан отрицательно определён)")
        elif np.any(eigenvalues > 0) and np.any(eigenvalues < 0):
            print("  → СЕДЛОВАЯ ТОЧКА (гессиан неопределён)")
        else:
            print("  → Неопределённый случай (нулевые собственные значения)")
    
    return hessian

def visualize_hessian_eigenvectors(hessian, point, f):
    """
    Визуализация собственных векторов гессиана
    
    Parameters:
    -----------
    hessian : ndarray
        Матрица Гессе 2x2
    point : tuple
        Точка анализа
    f : callable
        Исходная функция
    """
    # Вычисляем собственные векторы и значения
    eigenvalues, eigenvectors = np.linalg.eig(hessian)
    
    # Создаем сетку вокруг точки
    range_size = 0.5
    x = np.linspace(point[0]-range_size, point[0]+range_size, 30)
    y = np.linspace(point[1]-range_size, point[1]+range_size, 30)
    X, Y = np.meshgrid(x, y)
    
    # Значения функции
    Z = np.zeros_like(X)
    for i in range(30):
        for j in range(30):
            Z[i, j] = f([X[i, j], Y[i, j]])
    
    # Создаем фигуру
    fig = go.Figure()
    
    # Поверхность функции
    fig.add_trace(go.Surface(
        x=X, y=Y, z=Z,
        colorscale='Viridis',
        opacity=0.8,
        name='Функция'
    ))
    
    # Точка анализа
    f0 = f(point)
    fig.add_trace(go.Scatter3d(
        x=[point[0]], y=[point[1]], z=[f0],
        mode='markers',
        marker=dict(size=6, color='red'),
        name='Точка анализа'
    ))
    
    # Собственные векторы
    scale = 0.3
    colors = ['blue', 'green']
    
    for i in range(2):
        vec = eigenvectors[:, i] * scale
        # Проекция на плоскость XY
        fig.add_trace(go.Scatter3d(
            x=[point[0], point[0] + vec[0]],
            y=[point[1], point[1] + vec[1]],
            z=[f0, f0],  # В плоскости
            mode='lines+markers',
            line=dict(width=5, color=colors[i]),
            marker=dict(size=4),
            name=f'Собств. вектор {i+1} (λ={eigenvalues[i]:.3f})'
        ))
    
    # Обновляем layout
    fig.update_layout(
        title=f'Собственные векторы гессиана в точке {point}',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='f(x,y)',
            aspectmode='cube'
        ),
        height=700
    )
    
    fig.show()

# Пример использования
if __name__ == "__main__":
    print("Пример 1: Квадратичная форма")
    print("-" * 40)
    
    # Пример 1: Квадратичная форма
    A = np.array([[2, 1], [1, 3]])
    quad_func = lambda x: quadratic_form(np.array(x), A)
    
    point1 = (0.5, -0.5)
    hess1 = visualize_function_and_hessian(quad_func, point=point1)
    visualize_hessian_eigenvectors(hess1, point1, quad_func)
    
    print("\n" + "=" * 60)
    print("Пример 2: Функция Розенброка")
    print("-" * 40)
    
    # Пример 2: Функция Розенброка
    rosen_func = lambda x: rosenbrock(x, a=1, b=100)
    
    # Анализ в нескольких точках
    points_to_analyze = [(0, 0), (1, 1), (0.5, 0.25), (-1, 1)]
    
    for pt in points_to_analyze:
        print(f"\nАнализ в точке {pt}:")
        hess2 = approx_hessian(rosen_func, np.array(pt))
        eigenvalues = np.linalg.eigvals(hess2)
        print(f"  Собственные значения: {eigenvalues}")
    
    # Визуализация для одной точки
    point2 = (0.5, 0.25)
    hess2 = visualize_function_and_hessian(rosen_func, 
                                         x_range=(-2, 2), 
                                         y_range=(-1, 3), 
                                         point=point2)
    visualize_hessian_eigenvectors(hess2, point2, rosen_func)
    
    # Пример 3: Произвольная функция
    print("\n" + "=" * 60)
    print("Пример 3: Произвольная функция f(x,y) = sin(x) * cos(y)")
    print("-" * 40)
    
    custom_func = lambda x: np.sin(x[0]) * np.cos(x[1])
    point3 = (np.pi/4, np.pi/3)
    
    hess3 = visualize_function_and_hessian(custom_func, 
                                         x_range=(-np.pi, np.pi), 
                                         y_range=(-np.pi, np.pi), 
                                         point=point3)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# ---------- функция ----------
a, b = 1.0, 5.0

def grad(x):
    return np.array([a * x[0], b * x[1]])

H_true = np.array([[a, 0],
                   [0, b]])

# ---------- BFGS ----------
def bfgs_update(B, s, y):
    rho = 1.0 / (y @ s)
    I = np.eye(len(s))
    return (I - rho * np.outer(s, y)) @ B @ (I - rho * np.outer(y, s)) + rho * np.outer(s, s)

# ---------- оптимизация ----------
x = np.array([2.0, 2.0])
B = np.eye(2)
Bs = [B.copy()]

for _ in range(6):
    p = -np.linalg.solve(B, grad(x))
    x_new = x + 0.3 * p
    s = x_new - x
    y = grad(x_new) - grad(x)
    B = bfgs_update(B, s, y)
    Bs.append(B.copy())
    x = x_new

# ---------- сетка ----------
X = np.linspace(-2, 2, 70)
Y = np.linspace(-2, 2, 70)
XX, YY = np.meshgrid(X, Y)

def quad_surface(B):
    Z = np.zeros_like(XX)
    for i in range(XX.shape[0]):
        for j in range(XX.shape[1]):
            v = np.array([XX[i, j], YY[i, j]])
            Z[i, j] = 0.5 * v @ B @ v
    return Z

Z_true = quad_surface(H_true)

# ---------- кадры ----------
frames = []
for k, Bk in enumerate(Bs):
    frames.append(
        go.Frame(
            data=[
                go.Surface(
                    x=XX, y=YY, z=Z_true,
                    opacity=0.3,
                    colorscale="Viridis",
                    showscale=False
                ),
                go.Surface(
                    x=XX, y=YY, z=quad_surface(Bk),
                    opacity=0.9,
                    colorscale="Reds",
                    showscale=False
                )
            ],
            name=f"iter{k}"
        )
    )

# ---------- фигура ----------
fig = go.Figure(
    data=frames[0].data,
    frames=frames
)

fig.update_layout(
    title="BFGS: approximation of the Hessian (animation)",
    scene=dict(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="quadratic form",
    ),
    width=900,
    height=700,
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label="▶ Play",
                    method="animate",
                    args=[None, {"frame": {"duration": 800, "redraw": True},
                                 "fromcurrent": True}]
                ),
                dict(
                    label="⏸ Pause",
                    method="animate",
                    args=[[None], {"frame": {"duration": 0},
                                   "mode": "immediate"}]
                )
            ],
        )
    ]
)

fig.show()
